In [1]:
import numpy as np

In [2]:
def build_data(corpus, nm=30, stride=3):
    #nm being length of a training example, stride being window shift while scannig the corpus, returns X, Y for training
    #keeping nm 30 and stride 3
    X = []
    Y = []
    for i in range(0, len(corpus)-nm, stride):
        X.append(corpus[i: i+nm])
        Y.append(corpus[i+nm])
    print(f'number of training examples: {len(X)}')
    return X, Y

In [3]:
import io
corpus = io.open('1. corpus.txt', encoding='utf-8').read().lower()
chars = sorted(list(set(corpus))) #to sample the uniques characters from the corpus
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

nm = 30
stride = 3
X, Y = build_data(corpus, nm, stride)

number of training examples: 371788


In [4]:
def vectorization(X, Y, nv, char_indices, nm=30):
    #nv corresponds to length of a vector to be formed
    m = len(X)
    x = np.zeros((m, nm, nv), dtype=np.bool_)
    y = np.zeros((m, nv), dtype=np.bool_)
    for i, example in enumerate(X):
        for j, char in enumerate(example):
            x[i, j, char_indices[char]] = 1
        y[i, char_indices[Y[i]]] = 1
    return x, y

In [5]:
def sample(preds, temperature = 1.0):
    #sample so as to generate diverse literature
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)/temperature
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1) #sampling with n=1 experiment(s)
    sampled = np.random.choice(range(len(chars)), p = probs.ravel())
    return sampled

In [6]:
x, y = vectorization(X, Y, len(chars), char_indices)
print('vectorized, ready to use x, y')

vectorized, ready to use x, y


In [7]:
lvector = len(char_indices)
lx = len(x[0])
na = 128
ly = 1

In [8]:
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
from tensorflow.keras.models import Model

def models(lx, na, lvector):
    #model will generate only one character at a time stamp; lx(30x39), ly(39,) being lengths of inputs x, y
    X = Input(shape=(lx, lvector))
    a = LSTM(units=na, return_sequences=True)(X)
    a = Dropout(0.3)(a)
    a = LSTM(units=na, return_sequences=False)(a)
    a = Dropout(0.3)(a)
    out = Dense(lvector, activation='softmax')(a)
    model = Model(X, out)
    return model

model = models(lx, na, lvector)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30, 39)]          0         
                                                                 
 lstm (LSTM)                 (None, 30, 128)           86016     
                                                                 
 dropout (Dropout)           (None, 30, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 39)                5031      
                                                                 
Total params: 222,631
Trainable params: 222,631
Non-trainable

In [9]:
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.utils import to_categorical
#applying gradient clipping to prevent experienced eploding gradients
opt = Adam(learning_rate=0.07, beta_1=0.9, beta_2=0.999, decay=0.001, clipvalue=1.) 
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [10]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.05)

model.fit(xtrain, ytrain, epochs=2000, batch_size=512)

Epoch 1/2000
690/690 [==============================] - 13s 14ms/step - loss: 3.0589 - accuracy: 0.1593
Epoch 2/2000
690/690 [==============================] - 9s 14ms/step - loss: 3.0396 - accuracy: 0.1601
Epoch 3/2000
690/690 [==============================] - 9s 14ms/step - loss: 2.7504 - accuracy: 0.2116
Epoch 4/2000
690/690 [==============================] - 9s 14ms/step - loss: 2.5568 - accuracy: 0.2570
Epoch 5/2000
690/690 [==============================] - 9s 14ms/step - loss: 2.4765 - accuracy: 0.2791
Epoch 6/2000
690/690 [==============================] - 9s 14ms/step - loss: 2.3987 - accuracy: 0.2992
Epoch 7/2000
690/690 [==============================] - 9s 14ms/step - loss: 2.3346 - accuracy: 0.3145
Epoch 8/2000
690/690 [==============================] - 9s 14ms/step - loss: 2.2830 - accuracy: 0.3268
Epoch 9/2000
690/690 [==============================] - 10s 14ms/step - loss: 2.2381 - accuracy: 0.3390
Epoch 10/2000
690/690 [==============================] - 9s 14ms/step -

690/690 [==============================] - 9s 13ms/step - loss: 1.6555 - accuracy: 0.4923
Epoch 80/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.6528 - accuracy: 0.4932
Epoch 81/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.6503 - accuracy: 0.4933
Epoch 82/2000
690/690 [==============================] - 9s 13ms/step - loss: 1.6510 - accuracy: 0.4934
Epoch 83/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.6478 - accuracy: 0.4939
Epoch 84/2000
690/690 [==============================] - 10s 14ms/step - loss: 1.6476 - accuracy: 0.4936
Epoch 85/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.6460 - accuracy: 0.4951
Epoch 86/2000
690/690 [==============================] - 10s 14ms/step - loss: 1.6449 - accuracy: 0.4946
Epoch 87/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.6452 - accuracy: 0.4940
Epoch 88/2000
690/690 [==============================] - 9s 14ms/step - loss

690/690 [==============================] - 9s 13ms/step - loss: 1.5870 - accuracy: 0.5094
Epoch 158/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5858 - accuracy: 0.5098
Epoch 159/2000
690/690 [==============================] - 9s 13ms/step - loss: 1.5897 - accuracy: 0.5091
Epoch 160/2000
690/690 [==============================] - 9s 13ms/step - loss: 1.5861 - accuracy: 0.5098
Epoch 161/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.6054 - accuracy: 0.5038
Epoch 162/2000
690/690 [==============================] - 9s 13ms/step - loss: 1.5904 - accuracy: 0.5087
Epoch 163/2000
690/690 [==============================] - 9s 13ms/step - loss: 1.5869 - accuracy: 0.5098
Epoch 164/2000
690/690 [==============================] - 9s 13ms/step - loss: 1.5854 - accuracy: 0.5107
Epoch 165/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5829 - accuracy: 0.5103
Epoch 166/2000
690/690 [==============================] - 10s 14ms/ste

690/690 [==============================] - 9s 14ms/step - loss: 1.5614 - accuracy: 0.5160
Epoch 236/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5628 - accuracy: 0.5159
Epoch 237/2000
690/690 [==============================] - 9s 13ms/step - loss: 1.5622 - accuracy: 0.5154
Epoch 238/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5624 - accuracy: 0.5161
Epoch 239/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5599 - accuracy: 0.5162
Epoch 240/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5609 - accuracy: 0.5157
Epoch 241/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5610 - accuracy: 0.5165
Epoch 242/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5591 - accuracy: 0.5166
Epoch 243/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5599 - accuracy: 0.5164
Epoch 244/2000
690/690 [==============================] - 9s 14ms/step

690/690 [==============================] - 9s 14ms/step - loss: 1.5441 - accuracy: 0.5196
Epoch 314/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5444 - accuracy: 0.5204
Epoch 315/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5439 - accuracy: 0.5197
Epoch 316/2000
690/690 [==============================] - 10s 14ms/step - loss: 1.5433 - accuracy: 0.5208
Epoch 317/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5432 - accuracy: 0.5199
Epoch 318/2000
690/690 [==============================] - 9s 13ms/step - loss: 1.5439 - accuracy: 0.5206
Epoch 319/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5431 - accuracy: 0.5202
Epoch 320/2000
690/690 [==============================] - 9s 13ms/step - loss: 1.5412 - accuracy: 0.5211
Epoch 321/2000
690/690 [==============================] - 9s 14ms/step - loss: 1.5420 - accuracy: 0.5213
Epoch 322/2000
690/690 [==============================] - 9s 13ms/ste

...

690/690 [==============================] - 10s 15ms/step - loss: 1.4758 - accuracy: 0.5379
Epoch 1696/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4744 - accuracy: 0.5380
Epoch 1697/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4760 - accuracy: 0.5377
Epoch 1698/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4747 - accuracy: 0.5390
Epoch 1699/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4753 - accuracy: 0.5383
Epoch 1700/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4744 - accuracy: 0.5384
Epoch 1701/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4732 - accuracy: 0.5387
Epoch 1702/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4751 - accuracy: 0.5383
Epoch 1703/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4732 - accuracy: 0.5392
Epoch 1704/2000
690/690 [============================

690/690 [==============================] - 10s 15ms/step - loss: 1.4824 - accuracy: 0.5357
Epoch 1772/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4816 - accuracy: 0.5357
Epoch 1773/2000
690/690 [==============================] - 11s 15ms/step - loss: 1.4792 - accuracy: 0.5361
Epoch 1774/2000
690/690 [==============================] - 11s 16ms/step - loss: 1.4803 - accuracy: 0.5363
Epoch 1775/2000
690/690 [==============================] - 11s 16ms/step - loss: 1.4804 - accuracy: 0.5360
Epoch 1776/2000
690/690 [==============================] - 17s 25ms/step - loss: 1.4793 - accuracy: 0.5365
Epoch 1777/2000
690/690 [==============================] - 32s 47ms/step - loss: 1.4801 - accuracy: 0.5363
Epoch 1778/2000
690/690 [==============================] - 33s 48ms/step - loss: 1.4796 - accuracy: 0.5369
Epoch 1779/2000
690/690 [==============================] - 33s 48ms/step - loss: 1.4812 - accuracy: 0.5370
Epoch 1780/2000
690/690 [============================

690/690 [==============================] - 37s 53ms/step - loss: 1.4778 - accuracy: 0.5372
Epoch 1848/2000
690/690 [==============================] - 34s 49ms/step - loss: 1.4779 - accuracy: 0.5368
Epoch 1849/2000
690/690 [==============================] - 32s 47ms/step - loss: 1.4764 - accuracy: 0.5374
Epoch 1850/2000
690/690 [==============================] - 35s 50ms/step - loss: 1.4779 - accuracy: 0.5375
Epoch 1851/2000
690/690 [==============================] - 35s 51ms/step - loss: 1.4764 - accuracy: 0.5382
Epoch 1852/2000
690/690 [==============================] - 34s 50ms/step - loss: 1.4780 - accuracy: 0.5374
Epoch 1853/2000
690/690 [==============================] - 33s 47ms/step - loss: 1.4771 - accuracy: 0.5378
Epoch 1854/2000
690/690 [==============================] - 32s 46ms/step - loss: 1.4777 - accuracy: 0.5370
Epoch 1855/2000
690/690 [==============================] - 30s 43ms/step - loss: 1.4775 - accuracy: 0.5381
Epoch 1856/2000
690/690 [============================

690/690 [==============================] - 10s 15ms/step - loss: 1.4754 - accuracy: 0.5371
Epoch 1924/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4786 - accuracy: 0.5371
Epoch 1925/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4771 - accuracy: 0.5378
Epoch 1926/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4756 - accuracy: 0.5376
Epoch 1927/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4743 - accuracy: 0.5380
Epoch 1928/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4745 - accuracy: 0.5382
Epoch 1929/2000
690/690 [==============================] - 11s 15ms/step - loss: 1.4755 - accuracy: 0.5377
Epoch 1930/2000
690/690 [==============================] - 11s 16ms/step - loss: 1.4754 - accuracy: 0.5378
Epoch 1931/2000
690/690 [==============================] - 10s 15ms/step - loss: 1.4757 - accuracy: 0.5380
Epoch 1932/2000
690/690 [============================

690/690 [==============================] - 11s 15ms/step - loss: 1.4747 - accuracy: 0.5379
Epoch 2000/2000
690/690 [==============================] - 11s 15ms/step - loss: 1.4777 - accuracy: 0.5377


In [17]:
model.save('sakeswks.h5')

In [15]:
import sys

def generate(lit):
    #lit gives you option to choose how long your literature shall be (characterwise)
    generated = ''
    user_input = input('Initialize your literature and the Shakespeare will complete the rest, else just press enter: ')
    sentence = ('{0:0>' + str(nm) + '}').format(user_input).lower()
    ls = len(sentence)
    if ls>30:
        sentence = sentence[ls-nm:]
    generated+=user_input
    sys.stdout.write('\n\n Here is your poem: \n\n')
    sys.stdout.write(user_input) 
    for i in range(lit):
        x_pred = np.zeros((1, nm, len(chars)))
        for j, char in enumerate(sentence):
            if char!='0':
                x_pred[0, j, char_indices[char]] = 1
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature=1.0) #higher probability increases the probability of selecting a less likely character
        next_char = indices_char[next_index]
        generated+=next_char
        sentence = sentence[1:] + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
        if next_char=='\n':
            continue

In [16]:
generate(1024)

Initialize your literature and the Shakespeare will complete the rest, else just press enter: nor marching in the fields of tharsimen


 Here is your poem: 

nor marching in the fields of tharsiment,
and child he threstour hangart.

for tongelaus:
purest loons sens.

answard:
petnyst of the foult.

katharina:
they will sake theaters, i hear as father.

first pol

C:\Users\2jeet\AppData\Local\Temp\ipykernel_12344\502334462.py:4: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds)/temperature


ixherlay:
what is that hath death.

queen margaret:
remembitice bail and all is in lum no is should with yet he good,
to whose dead, strike puigmases?

polixenes:

farsius:
the fide! his alrscice, i will here from then;
should precaoub celless in yearlly some cofly and grief;
no rad king king edward for a found he king as titl;
rirfillio and a, igward distir;
you harses his fatast of him have many'd and
the rea?

duke of york:
no mustigg, silcorabtestron's king feath.

wither:

henry bolingbroke:
and goees quand moon in my light.

queen elizabes:
go other: a grains this barirg,
as mer, speak to the dofine enterven!
what manise thou three hope of erge,
and her
to me made all compenin savels! and, musy, providies'
death to lest sweet 'tis leg may.

vorumnia:
give them,---
that now, see, the lees condect sound upon;
and wall my all a timion, sives,